#### Importing necessary libraries

In [35]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


In [36]:
# Load the dataset and preprocess
df = pd.read_csv("preprocessed_dataset.csv")
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

In [37]:
df.head()

,% Iron Feed,% Silica Feed,Starch Flow,Amina Flow,Ore Pulp Flow,Ore Pulp pH,Ore Pulp Density,Flotation Column 01 Air Flow,Flotation Column 02 Air Flow,Flotation Column 03 Air Flow,...,Flotation Column 06 Air Flow,Flotation Column 07 Air Flow,Flotation Column 01 Level,Flotation Column 02 Level,Flotation Column 03 Level,Flotation Column 04 Level,Flotation Column 05 Level,Flotation Column 06 Level,Flotation Column 07 Level,% Silica Concentrate
date,,,,,,,,,,,,,,,,,,,,,
2017-03-10 01:03:00,56.39,13.53,2082.27,391.733,403.738,9.70067,1.74925,250.509,251.163,250.610,...,250.389,249.430,606.023,596.422,604.776,437.700,430.352,433.920,443.036,1.40
2017-03-10 01:03:20,55.20,16.98,3019.53,557.434,395.713,10.06640,1.74000,249.214,253.235,250.576,...,250.225,250.884,457.396,432.962,424.954,443.558,502.255,446.370,523.344,1.31
2017-03-10 01:03:40,55.20,16.98,3024.41,563.965,397.383,10.06720,1.74000,249.719,250.532,250.862,...,250.137,248.994,451.891,429.560,432.939,448.086,496.363,445.922,498.075,1.31
2017-03-10 01:04:00,55.20,16.98,3043.46,568.054,399.668,10.06800,1.74000,249.741,247.874,250.313,...,251.345,248.071,451.240,468.927,434.610,449.688,484.411,447.826,458.567,1.31
2017-03-10 01:04:20,55.20,16.98,3047.36,568.665,397.939,10.06890,1.74000,249.917,254.487,250.049,...,250.422,251.147,452.441,458.165,442.865,446.210,471.411,437.690,427.669,1.31


#### Useful variables selection

In [38]:
print("TODO")

TODO


#### PLS loop

In [33]:
from sklearn.preprocessing import StandardScaler

# Reduce the amount of observations (with averaging)
df = df.resample("T").mean()

included_rows1 = [df.index.get_loc(datetime.datetime(2017,4,23,19,0,0)),
                    df.index.get_loc(datetime.datetime(2017,5,13,0,59,00))]
included_rows2 = [df.index.get_loc(datetime.datetime(2017,6,15,1,0,0)),
                    df.index.get_loc(datetime.datetime(2017,7,24,0,59,00))]
included_rows3 = [df.index.get_loc(datetime.datetime(2017,8,15,1,0,0)),
                    df.index.get_loc(datetime.datetime(2017,9,10,0,0,0))]

df_array = df.to_numpy()

# Find best window_size and num_of_components
for window_size in [20]:  # [10, 20, 30, ...]
    for num_of_components in [7]:  # range(1, len(df.columns)):
        # Division of data into training, validation and test sets for the normal functioning times:
        # about 70% for training, 15% for validation, 15% for testing
        test_size = round(0.15*window_size)
        test_start = '2017-04-23 00:00:00' # example start of test partition
        test_start = datetime.datetime.strptime(test_start, '%Y-%m-%d %H:%M:%S')
        flag = 0

        row_number = df.index.get_loc(test_start)

        # Iterate through the data
        while flag == 0:
            test_y_end = row_number+test_size+1
            training_X_start = row_number - round(0.85*window_size)
            # Stay within included rows
            if ((test_y_end <= included_rows1[1] and training_X_start >= included_rows1[0])
            or (test_y_end <= included_rows2[1] and training_X_start >= included_rows2[0])
            or (test_y_end <= included_rows3[1] and training_X_start >= included_rows3[0])):

                scaler = StandardScaler()

                # Validation set
                validation_X = df_array[row_number-test_size:row_number,:-1]
                validation_y = df_array[row_number-test_size+1:row_number+1,-1]

                # Training set
                training_X = df_array[training_X_start:row_number-test_size,:-1]
                training_y = df_array[training_X_start+1:row_number-test_size+1,-1]

                # Lagged predictor variables and the response variable in one matrix
                validation_data = np.column_stack([validation_X, validation_y])
                training_data = np.column_stack([training_X, training_y])

                # Compute the mean and std with training data
                scaler.fit(training_data)
                # Perform standardization
                training_data = scaler.transform(training_data)
                validation_data = scaler.transform(validation_data)

                if test_y_end == included_rows3[1]:
                    flag = 1

                row_number = row_number + 1
            else:
                if test_y_end > included_rows3[1]:
                    flag = 1

                row_number = row_number + 1

                # # TODO model fitting
                # for _ in [1]:
                #     ...

                # # TODO model testing
                # for _ in [1]:
                #     ...
        

In [ ]:
# Test model

window_size = 20  # best window_size
num_of_components = 7 # best num_of_components

test_size = round(0.15*window_size)
test_start = '2017-04-23 00:00:00' # example start of test partition
test_start = datetime.datetime.strptime(test_start, '%Y-%m-%d %H:%M:%S')
flag = 0

row_number = df.index.get_loc(test_start)

# Iterate through the data
while flag == 0:
    test_y_end = row_number+test_size+1
    training_X_start = row_number - round(0.85*window_size)
    # Stay within included rows
    if ((test_y_end <= included_rows1[1] and training_X_start >= included_rows1[0])
    or (test_y_end <= included_rows2[1] and training_X_start >= included_rows2[0])
    or (test_y_end <= included_rows3[1] and training_X_start >= included_rows3[0])):

        scaler = StandardScaler()

        # Testing set
        test_X = df_array[row_number:test_y_end-1,:-1]
        test_y = df_array[row_number+1:test_y_end,-1]

        # Training (+ validation) set
        training_X = df_array[training_X_start:row_number,:-1]
        training_y = df_array[training_X_start+1:row_number+1,-1]

        # Lagged predictor variables and the response variable in one matrix
        test_data = np.column_stack([test_X, test_y])
        training_data = np.column_stack([training_X, training_y])

        # Compute the mean and std with training data
        scaler.fit(training_data)
        # Perform standardization
        training_data = scaler.transform(training_data)
        test_data = scaler.transform(test_data)

        if test_y_end == included_rows3[1]:
            flag = 1

        row_number = row_number + 1
    else:
        if test_y_end > included_rows3[1]:
            flag = 1

        row_number = row_number + 1

        # # TODO model fitting
        # for _ in [1]:
        #     ...

        # # TODO model testing
        # for _ in [1]:
        #     ...
        